# Homework Assignment #4 (Individual)
## Regression and SVM Predictive Models

### <p style="text-align: right;"> &#9989; Hanwen Zuo</p>
### <p style="text-align: right;"> &#9989; zuohanwe</p>

### Goal for this homework assignment
By now, you have learned a bit about predictive models. In this assignment, you will practice:

* Building predictive models using `statsmodels` and `sklearn`
* Evaluating your predictive models

**This assignment is due roughly two weeks from now at 11:59 pm on Friday, April 21st.** It should be uploaded into the "Homework Assignments" submission folder for Homework #4.  Submission instructions can be found at the end of the notebook.


---
## Part 0: Add to your Git repository to track your progress on your assignment (4 points)

For this assignment, you're going to add it to the `cmse202-s23-turnin` repository you created in class so that you can track your progress on the assignment and preserve the final version that you turn in. In order to do this you need to

**&#9989; Do the following**:

1. Navigate to your `cmse202-s23-turnin` repository and create a new directory called `hw-04`.
2. Move this notebook into that **new directory** in your repository, then **add it and commit it to your repository**.
1. Finally, to test that everything is working, "git push" the file so that it ends up in your GitHub repository.

**Important**: Double check you've added your Professor and your TA as collaborators to your "turnin" repository (you should have done this in the previous homework assignment).

**Also important**: Make sure that the version of this notebook that you are working on is the same one that you just added to your repository! If you are working on a different copy of the notebook, **none of your changes will be tracked**!

If everything went as intended, the file should now show up on your GitHub account in the "`cmse202-s23-turnin`" repository inside the `hw-04` directory that you just created.  Periodically, **you'll be asked to commit your changes to the repository and push them to the remote GitHub location**. Of course, you can always commit your changes more often than that, if you wish.  It can be good to get into a habit of committing your changes any time you make a significant modification, or when you stop working on the project for a bit.

&#9989; **Do this**: Before you move on, put the command that your instructor should run to clone your repository in the markdown cell below.

`git clone https://github.com/zuohanwe/CMSE202-s23-turnin.git`

# Building a Model to Predict if it Will Rain Tomorrow

In this dataset we have weather data for a number of Australian cities. We’re going to **build a model that uses data from today to predict if it will rain tomorrow.** To simplify our model, let’s just look at the weather for one city.

## Part 1. Working with the Data

The dataset that we’ll be using can be found here:
- `https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv`

#### 1.1 (3 Points)
**Download the data set and read it in using Pandas.**

In [19]:
### Imports
# 3rd party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

In [2]:
#Write your code here
# curl --remote-name 'https://raw.githubusercontent.com/msu-cmse-courses/cmse202-S23-data/main/HW/HW4/aussie_weather_data.csv'

weather_df = pd.read_csv('aussie_weather_data.csv')

#### 1.2 (3 Points)
This dataset has weather information for multiple cities in Australia. It’s quite a large dataset. In fact, it’s a little *too* large for our purposes; it will take a considerable amount of time to train a model on so much data. So we’ll just work with the data for a specific city.

The first thing we’ll do is see which cities we have data for. 
**Print out a list of the specific cities in this dataset.** (Note that this is not the same as printing the entire `Location` column.)

In [3]:
#Write your code here
print(weather_df['Location'].unique())

['NorfolkIsland' 'SydneyAirport' 'WaggaWagga' 'MelbourneAirport' 'Mildura'
 'Watsonia' 'Brisbane' 'Cairns' 'Townsville' 'MountGambier' 'Nuriootpa'
 'PerthAirport' 'Perth' 'AliceSprings' 'Darwin']


#### 1.3 (3 Points)
Select one of the cities (and **please** don't just pick the first one!). Cut down your dataframe so that it only contains data from your chosen city. You should also **look through the columns in this dataset and ensure that all of the data is ready for our analysis/model creation.**

In [4]:
#Write your code here
city_weather = weather_df[weather_df['Location'] == 'Nuriootpa']
x_data = city_weather.drop(columns=['RainTomorrow', 'Location'])
y_data = city_weather['RainTomorrow']

In [5]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2008 entries, 26284 to 28291
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year           2008 non-null   int64  
 1   Month          2008 non-null   int64  
 2   DayOfMonth     2008 non-null   int64  
 3   MinTemp        2008 non-null   float64
 4   MaxTemp        2008 non-null   float64
 5   Rainfall       2008 non-null   float64
 6   Evaporation    2008 non-null   float64
 7   Sunshine       2008 non-null   float64
 8   WindGustDir    2008 non-null   float64
 9   WindGustSpeed  2008 non-null   float64
 10  WindDir9am     2008 non-null   float64
 11  WindDir3pm     2008 non-null   float64
 12  WindSpeed9am   2008 non-null   float64
 13  WindSpeed3pm   2008 non-null   float64
 14  Humidity9am    2008 non-null   float64
 15  Humidity3pm    2008 non-null   float64
 16  Pressure9am    2008 non-null   float64
 17  Pressure3pm    2008 non-null   float64
 18  Clo

#### 1.4 (3 Points)
Finally, before we start creating our models, let's first split our data into training and testing datasets. **Keep in mind what we want our model to predict and how this dataset gives you the information you need for your model.**

In [14]:
#Write your code here
train_vectors, test_vectors, train_labels, test_labels = train_test_split(x_data, y_data, random_state=1)

## 2.0 Building a Regression Model

#### 2.1 (3 Points)
We’ll start by creating a regression model. **Does it make sense for us to use linear regression or logistic regression? Explain your choice.**

A logistic model makes more sense when we're trying to predict a binary outcome of rain or no rain. It wouldn't make sense to use a linear regression because it gives continuous values rather than a prediction.

#### 2.2 (12 Points)
Create a regression model. Use the model summary to determine which parameters are most statistically significant in your model. If you get an error during at this stage it may be helpful to go back to part 1.3 and ensure that all of your data is appropriate for creating a model. 

In [15]:
#Write your code here
logit_model = sm.Logit(train_labels, train_vectors)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.258939
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           RainTomorrow   No. Observations:                 1506
Model:                          Logit   Df Residuals:                     1483
Method:                           MLE   Df Model:                           22
Date:                Fri, 21 Apr 2023   Pseudo R-squ.:                  0.4694
Time:                        22:37:46   Log-Likelihood:                -389.96
converged:                       True   LL-Null:                       -734.94
Covariance Type:            nonrobust   LLR p-value:                1.019e-131
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Year              0.0517      0.010      5.216      0.000       0.032       0.071
Month             0.

Here are the significant parameters (p < 0.05) listed from most to least significant: `WindGustSpeed, Year, Sunshine, Pressure3pm, WindGustDir, Humidity3pm, Cloud3pm`.

#### 2.3 (8 Points)
**Fit your test data and create/print a confusion matrix. We’ll use this to evaluate how well your predicitve model performs.**

In [18]:
#Write your code here
predicted_y = [round(x) for x in result.predict(test_vectors)]
confusion_matrix(test_labels, predicted_y)

array([[381,  20],
       [ 46,  55]])

In [21]:
print(precision_score(test_labels, predicted_y))
print(recall_score(test_labels, predicted_y))

0.7333333333333333
0.5445544554455446


#### 2.4 (3 Points)
Look at/calculate the Precision and Recall of your model. **Which one is greater? Describe in plain language what that means about the performance of your model (I.e., the circumstances in which it does/doesn’t do well).**

Precision is higher (0.73) than recall (0.54). Precision measures the percentage of times we predicted rain and actually got rain, and recall measures how many rainy days we correctly predicted out of all the rainy days.

If our model predicts rain, it has a roughly 70% chance of being correct. In other words, its predictions are reasonly precise; however, it has a recall of around 50%, which means out of all the rainy days it predicted rain only about 50% of the time. Thus, our model is likely to be correct when it predicts rain, but it only predicts about half of all rainy days.

## 3.0 Interlude: A Base Rate Frequency Model (12 Points)

If we didn't create a fancy model for predicting future rainfall, would that mean that we had *no way* of predicting if it would rain? Of course not. There's always *some* kind of model we can use (they just won't be very sophisticated). 

In the case of our problem--predicting whether it will rain tomorrow--what is the simplest model we could use? We'd need to look at the **base rate frequency**. If we know that, over the course of the year, it rains ~30% of the time, that would be our base rate frequency. If we were to guess that it would be sunny every day of the year, we'd be wrong ~30% of the time, *but we'd be right ~70% of the time.* 

Why is this important? Imagine we create a sophisticated model that accurately predicts rain/no rain ~60% of the time; without any other information, we might think, "That's not too bad, I guess." It certainly feels better than *no* information, right? Now think about the base rate frequency model, which achieves an accuracy of **70%** just by guessing the same thing every time. Suddenly, our sophisticated model seems like crap! It tells us we could be doing ~10% better than our fancy model by guessing. This is why we use the base rate frequency as a simple way to test our models.


**Calculate how frequently it rains in your city. If you were to guess that it *wasn't* going to rain every day, how frequently would you be right? (We'll use this as our baseline model.)**


In [25]:
#Write your code for calculating the frequency of rain here.
no_rain, rain = city_weather.RainToday.value_counts()
print(no_rain / city_weather.shape[0])

0.8087649402390438


If we predicted no rain, we'd be correct about 80% of the time.

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

## 4.0 SVM Model

We’ve seen how regression faired in predicting rain; now let’s give Support Vector Machines a shot. 

It’s not clear, *a priori*, which type of kernel (linear or RBF) would fair better. Therefore, we’ll need to test out both kernel types, as well as multiple values for hyperparameters ($C$ and $\gamma$). 

#### 4.1 (12 Points)
Run a grid search over both types of kernels and multiple values of $C$ and $\gamma$ (we recommend [0.01,0.1,1.0,10.0] for both). We *STRONGLY* recommend you set the parameter `n_jobs=-1` in `GridSearchCV`, which will distribute the computational load. Make sure to write down your best-fit hyperparameters. 

In [ ]:
#Write your code here

*Write down your best fir hyper-parameters here.*

#### 4.2 (8 Points)
Using your best-fit parameters, classify your test data and print off the confusion matrix. Note/calculate the Precision and Recall for your SVM model, as well as the overall accuracy.

$\mathrm{Accuracy} = \frac{\mathrm{Accurate~Predictions}}{\mathrm{Total~Samples}}$.

In [ ]:
#Write your code here

## 5. Evaluation of Models

Here we’re going to investigate how well our predictive models actually work.


#### 5.1 (5 Points)

Compare the accuracy of both of your predictive models (regression and SVM) to the baseline model from part 3. Do your predictive models do better than the baseline model? Is their performance relative to the baseline model what *you* would expect? 


*Write down your response here.*

#### 5.2 (5 Points)

Let’s test the robustness of your model. Before you do anything, you should record the precision, recall, and accuracy of your two predictive models (if you haven’t already). Once you’ve done that, go all the way back to part 1.4 and change the random seed for `train_test_split`. Using this new data split, rerun all of your code from parts 2 and 4. For each new random seed, record the precision, recall, and accuracy for both models (and write them, as well as the random seed, in the cell below!). Do this for 3-5 different random seeds. 


*Write down the accuracy, precision, recall, and seed number for each of your iterations.*

#### 5.3 (5 Points)

How much variation did you observe in the precision, recall, and accuracy in your models? After going through this exercise, how (if at all) does this change your response to part 5.1? 

*Write down your response here.*

#### 5.4 (5 Points)

Finally, compare the performance of your regression and SVM models. Which, if any, performed better? Esxplain.

*Write down your response here.*

---

### &#128721; STOP (3 Point)
**Pause to commit your changes to your Git repository!**

Take a moment to save your notebook, commit the changes to your Git repository with a meaningful commit message.

---

### Congratulations, you're done!

&#169; Copyright 2022,  Department of Computational Mathematics, Science and Engineering at Michigan State University